In [ ]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

In [62]:
print(torch.__version__)
print(f"MPS is available: {torch.backends.mps.is_built()}")

if torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

2.8.0
MPS is available: True


For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

* The shape for input should be `(3, 200, 200)` (channels first format in PyTorch)
* Next, create a convolutional layer (`nn.Conv2d`):
    * Use 32 filters (output channels)
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling (`nn.MaxPool2d`)
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using `flatten` or `view`
* Next, add a `nn.Linear` layer with 64 neurons and `'relu'` activation
* Finally, create the `nn.Linear` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use `torch.optim.SGD` with the following parameters:

* `torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)`

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=32, kernel_size=(3, 3))
        self.relu = nn.ReLU()

        self.pool = nn.MaxPool2d(kernel_size=(2, 2))

        self.fc1 = nn.Linear(32*99*99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)

        x=self.pool(x)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        
        return x

model = CNN()

model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# Q1

Which loss function you will use?

* `nn.BCEWithLogitsLoss()`

In [64]:
criterion = nn.BCEWithLogitsLoss()

# Q2

What's the total number of parameters of the model? You can use `torchsummary` or count manually. 

In PyTorch, you can find the total number of parameters using:

```python
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
```


In [65]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


For the next two questions, use the following transformation for both train and test sets:

```python
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
```

* We don't need to do any additional pre-processing for the images.
* Use `batch_size=20`
* Use `shuffle=True` for both training, but `False` for test. 

Now fit the model.

In [66]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader


train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

train_dataset = datasets.ImageFolder(
    root='./data/train',
    transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

validation_dataset = datasets.ImageFolder(
    root='./data/test',
    transform=train_transforms
)
validation_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [67]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6704, Acc: 0.6238, Val Loss: 0.5899, Val Acc: 0.6667
Epoch 2/10, Loss: 0.5560, Acc: 0.7000, Val Loss: 0.6891, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5532, Acc: 0.7175, Val Loss: 0.6033, Val Acc: 0.6567
Epoch 4/10, Loss: 0.4968, Acc: 0.7625, Val Loss: 0.7035, Val Acc: 0.6468
Epoch 5/10, Loss: 0.4641, Acc: 0.7688, Val Loss: 0.5825, Val Acc: 0.7065
Epoch 6/10, Loss: 0.3624, Acc: 0.8363, Val Loss: 0.6544, Val Acc: 0.6667
Epoch 7/10, Loss: 0.2754, Acc: 0.8925, Val Loss: 0.9617, Val Acc: 0.6567
Epoch 8/10, Loss: 0.2741, Acc: 0.8838, Val Loss: 0.6387, Val Acc: 0.7363
Epoch 9/10, Loss: 0.1848, Acc: 0.9337, Val Loss: 0.6227, Val Acc: 0.7363
Epoch 10/10, Loss: 0.1833, Acc: 0.9325, Val Loss: 0.7320, Val Acc: 0.7313


# Q3

What is the median of training accuracy for all the epochs for this model?


In [68]:
np.median(np.asarray(history['acc']))

np.float64(0.8025)

# Q4

What is the standard deviation of training loss for all the epochs for this model?

In [69]:
np.std(np.asarray(history['loss']))

np.float64(0.16136315074420945)

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

```python
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),
```

In [70]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )  # ImageNet normalization
])

train_dataset = datasets.ImageFolder(
    root='./data/train',
    transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

# Q5

Let's train our model for 10 more epochs using the same code as previously.

> **Note:** make sure you don't re-create the model.
> we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

In [71]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6999, Acc: 0.6362, Val Loss: 0.6217, Val Acc: 0.6567
Epoch 2/10, Loss: 0.5834, Acc: 0.6450, Val Loss: 0.7396, Val Acc: 0.6915
Epoch 3/10, Loss: 0.5640, Acc: 0.7037, Val Loss: 0.6431, Val Acc: 0.6816
Epoch 4/10, Loss: 0.5437, Acc: 0.7262, Val Loss: 0.5615, Val Acc: 0.7214
Epoch 5/10, Loss: 0.5450, Acc: 0.7087, Val Loss: 0.5837, Val Acc: 0.6965
Epoch 6/10, Loss: 0.5203, Acc: 0.7375, Val Loss: 0.5401, Val Acc: 0.7363
Epoch 7/10, Loss: 0.5317, Acc: 0.7188, Val Loss: 0.6984, Val Acc: 0.6667
Epoch 8/10, Loss: 0.5133, Acc: 0.7488, Val Loss: 0.7105, Val Acc: 0.6667
Epoch 9/10, Loss: 0.4926, Acc: 0.7438, Val Loss: 0.5147, Val Acc: 0.7363
Epoch 10/10, Loss: 0.4630, Acc: 0.7788, Val Loss: 0.5233, Val Acc: 0.7363


In [72]:
np.mean(np.asarray(history['val_loss']))

np.float64(0.6136488796466618)

# Q6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

In [73]:
np.mean(np.asarray(history['val_acc'])[-5:])

np.float64(0.708457711442786)